In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from datetime import datetime
import random
import cassandra
import datetime
from cassandra.util import uuid_from_time


In [2]:
CASSANDRA_HOST = "etl_cassandra"
CASSANDRA_KEYSPACE = "de_project"
CASSANDRA_TABLE = "tracking"

MYSQL_HOST = "localhost"
MYSQL_DATABASE = "Data_Warehouse"
# MYSQL_ROOT_PASSWORD = "123456"


In [3]:
spark = (
    SparkSession.builder
    .config("spark.jars.packages", 
            "com.datastax.spark:spark-cassandra-connector_2.12:3.1.0,"
            "com.mysql:mysql-connector-j:8.0.33")
    .getOrCreate()
)

In [4]:
df_tracking = spark.read.format("org.apache.spark.sql.cassandra").options(table=CASSANDRA_TABLE,keyspace=CASSANDRA_KEYSPACE)\
    .load()

In [5]:
from cassandra.cluster import Cluster

cluster = Cluster()  
session = cluster.connect()
session.set_keyspace(CASSANDRA_KEYSPACE)

In [10]:
df_tracking.select('job_id').distinct().show()

+------+
|job_id|
+------+
|  1533|
|   336|
|   122|
|   258|
|  1530|
|  1529|
|  1527|
|  1531|
|   273|
|  1103|
|    98|
|  1532|
|   188|
|   357|
|  1534|
|   187|
|  1280|
|   285|
|   493|
|   440|
+------+
only showing top 20 rows



In [ ]:
col_jobid = [496, 1533, 606, 336, 1197, 1400, 440, 760, 2026, 258, 1530, 1529, 1527, 1677, 1531, 273, 1103, 98, 1532, 188, 1981, 1534, 996, 187, 463, 192, 1130, 339, 1632, 179, 1893, 1592, 90, 1686, 213, 119, 1109, 737, 1871, 1602, 311, 1280, 285, 493, 1119, 197, 2009, 990, 379, 1710, 122, 357, 1005, 1886, 355, 1488, 1391, 1086, 858, 1156, 2051, 1447, 1681, 1644, 1218, 1196, 1634, 1723, 206, 263, 538, 0, 1539, 769, 125, 2002, 974, 512, 885, 1414, 1210, 1945, 672, 731, 1688, 1656, 1565, 671, 81, 76, 1806, 1497, 679, 247, 245, 782, 1285, 252, 458, 1616, 1822, 969, 505, 1059, 1343, 1257, 1430, 1078, 1997, 926, 1663, 212, 1169, 952, 481, 1144, 1849, 1791, 272, None]
col_customtrack = ['conversion', 'redirect', 'click', 'unqualified', 'alive', 'qualified', 'interview_scheduled', '0', 'superpixel', None]
col_bid = [1, 7, 2, 0, None]
col_campaignid = [222, 93, 232, 1, 13, 57, 48, 15, 4, 151, 79, 144, 122, 58, 12, 61, 53, 141, 33, 5, 63, 97, 101, 178, 54, 55, 193, 223, 59, 117, 9, 51, 70, 8, 10, 198, None]
col_groupid = [34, 28, 27, 16, 19, 10, 21, 11, 22, 13, 41, 32, 12, 15, 14, 35, 25, 17, 37, 26, 31, None]
col_publisherid = [1, 5, 3, 20, 15, 35, 33, 12, 22, 23, 11, 13, 9, 30, 27, 17, 34, 16, 24, 29, 2, 38, 31, 37, 10, 32, 36, 28, None]


In [ ]:
col_jobid = df_tracking.select('job_id').distinct().rdd.flatMap(lambda x: x).collect()
col_customtrack = df_tracking.select('custom_track').distinct().rdd.flatMap(lambda x: x).collect()
col_bid = df_tracking.select('bid').distinct().rdd.flatMap(lambda x: x).collect()
col_campaignid = df_tracking.select('campaign_id').distinct().rdd.flatMap(lambda x: x).collect()
col_groupid = df_tracking.select('group_id').distinct().rdd.flatMap(lambda x: x).collect()
col_publisherid = df_tracking.select('publisher_id').distinct().rdd.flatMap(lambda x: x).collect()


In [ ]:
records = int(input('Nhap so record fake: '))

for i in range(records):
    create_time = str(uuid_from_time(datetime.datetime.now()))
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    job_id = random.choice(col_jobid) or 0 
    custom_track = random.choice(col_customtrack) or 0 
    bid = random.choice(col_bid) or 0 
    campaign_id = random.choice(col_campaignid) or 0 
    group_id = random.choice(col_groupid) or 0 
    publisher_id = random.choice(col_publisherid) or 0 



    sql = """ INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ({},{},{},'{}',{},{},{},'{}')""".format(create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts)
    print(sql)
    session.execute(sql)
    print("INSERT INTO Cassandra thành công!", i)

# Đóng kết nối
cluster.shutdown()

 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES (38c42260-e630-11ef-ad38-793a1fc119eb,2,223,'unqualified',32,188,32,'2025-02-08 15:20:25')
INSERT INTO Cassandra thành công! 0
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES (38d455cc-e630-11ef-87d5-a58c08c88e2d,7,222,'interview_scheduled',26,0,13,'2025-02-08 15:20:25')
INSERT INTO Cassandra thành công! 1
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES (38d5d0be-e630-11ef-bbb1-250fb3d88090,2,97,'superpixel',19,1103,35,'2025-02-08 15:20:25')
INSERT INTO Cassandra thành công! 2
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES (38d7fda8-e630-11ef-9a97-094a9dac16aa,7,63,'redirect',27,0,33,'2025-02-08 15:20:25')
INSERT INTO Cassandra thành công! 3
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_